In [147]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

In [186]:
data = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv').reset_index()
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [149]:
X = data.drop('label', axis=1)
y = data['label']
yCat = to_categorical(y)

In [150]:
X_train,X_valid, y_train,y_valid = train_test_split(X,y, test_size=0.25)

In [151]:
model = SVC(kernel = 'rbf')
model.fit(X_train, y_train )

pred = model.predict(X_valid)
accuracy_score(y_valid, pred)

0.9773333333333334

In [153]:
X_train,X_valid, y_train,y_valid = train_test_split(X,yCat, test_size=0.25)

In [154]:
input_shape = (784,)

model = keras.Sequential([
    layers.Dense(512, activation = 'relu', input_shape = input_shape),
    layers.Dropout(0.25),

    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.25),
    
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.25),
    
    layers.Dense(128,activation = 'relu'),
    layers.Dropout(0.25),
    
    layers.Dense(10,activation = 'softmax')
])
model.summary()

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_114 (Dense)           (None, 512)               401920    
                                                                 
 dropout_20 (Dropout)        (None, 512)               0         
                                                                 
 dense_115 (Dense)           (None, 256)               131328    
                                                                 
 dropout_21 (Dropout)        (None, 256)               0         
                                                                 
 dense_116 (Dense)           (None, 256)               65792     
                                                                 
 dropout_22 (Dropout)        (None, 256)               0         
                                                                 
 dense_117 (Dense)           (None, 128)             

In [157]:
model.compile(optimizer=keras.optimizers.Adam(epsilon=0.01),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

In [158]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(x = X_train, y = y_train, validation_data=(X_valid, y_valid),epochs = 100, 
    verbose=1,batch_size=512, callbacks=[early_stopping])

Epoch 1/100
62/62 [==============================] - 1s 11ms/step - loss: 0.0774 - accuracy: 0.9769 - val_loss: 0.1232 - val_accuracy: 0.9714
Epoch 2/100
62/62 [==============================] - 1s 9ms/step - loss: 0.0747 - accuracy: 0.9788 - val_loss: 0.1261 - val_accuracy: 0.9706
Epoch 3/100
62/62 [==============================] - 1s 9ms/step - loss: 0.0723 - accuracy: 0.9780 - val_loss: 0.1275 - val_accuracy: 0.9716
Epoch 4/100
62/62 [==============================] - 1s 9ms/step - loss: 0.0759 - accuracy: 0.9776 - val_loss: 0.1261 - val_accuracy: 0.9697
Epoch 5/100
62/62 [==============================] - 1s 9ms/step - loss: 0.0735 - accuracy: 0.9778 - val_loss: 0.1263 - val_accuracy: 0.9716
Epoch 6/100
62/62 [==============================] - 1s 9ms/step - loss: 0.0721 - accuracy: 0.9787 - val_loss: 0.1271 - val_accuracy: 0.9713
Epoch 7/100
62/62 [==============================] - 1s 9ms/step - loss: 0.0716 - accuracy: 0.9792 - val_loss: 0.1206 - val_accuracy: 0.9734
Epoch 8/100


In [159]:
pred = model.predict(X_valid)

329/329 [==============================] - 0s 769us/step


In [162]:
res = np.argmax(pred, axis=1)
res

array([4, 7, 2, ..., 7, 6, 5])

In [170]:
resTest = np.argmax(model.predict(test.values), axis=1)
resTest

875/875 [==============================] - 1s 769us/step


array([2, 0, 9, ..., 3, 9, 2])

In [180]:
resTest[:]

array([2, 0, 9, ..., 3, 9, 2])

In [192]:
dfSubmission = pd.DataFrame({"ImageId":range(1,28001), "Label":resTest[:]}).set_index("ImageId")

In [193]:
dfSubmission.head()

,Label
ImageId,
1,2
2,0
3,9
4,9
5,3


In [194]:
dfSubmission.to_csv('submission.csv')